# Anti-Aliasing Filter Design

# Single-ended output filter
![](images/adc_rlc_filter.png)

Transfer function becomes: 

$$
\frac{V_O}{V_I}
=
\frac{\displaystyle \frac{1}{L C}}
     {s^2
      + s\Bigl(\frac{1}{R_2 C} + \frac{R_1}{L}\Bigr)
      + \Bigl(1 + \frac{R_1}{R_2}\Bigr)\frac{1}{L C}
     }$$

$$
\omega_O \;=\; \sqrt{\frac{1}{\alpha\,L\,C}}
$$

$$
Q \;=\;
\frac{
  \displaystyle \sqrt{\frac{1}{\alpha\,L\,C}}
}{
  \displaystyle \frac{1}{\alpha\,R_T\,C}
  \;+\;
  \frac{R_T\,(1 - \alpha)}{L}
}
$$

### R2
- Additional tuning parameter (Q-factor)
- DC bias current provision

Choose R2 = 2k (larger than 100 to avoid messing too much with the differential impedance) (1/100 + 1/2000)^(-1) ~= 100

In [1]:

import math 
#! >>>> CHOICE
R2_filter = 2e3
R1_filter = 5
w0 = 2*math.pi * 15e6
#! <<<< CHOICE

Rt = R1_filter + R2_filter
a_atten = (R2_filter) / (R1_filter + R2_filter)
Q_target = 0.707

print(f"DC-Attenuation: {a_atten}")
LC_product = (1 / (w0 * math.sqrt(a_atten)))**2
print(f"LC-product: {LC_product}")
L_filter = Rt / (2*w0*Q_target) * (1+math.sqrt(2*a_atten-1))
L_filter_no_R2 = Rt / (w0 * Q_target)
print(f"L_filter: {L_filter*1e9:.2f} nH or {L_filter_no_R2*1e9:.2f} nH")
C_filter = (1 / (a_atten* L_filter_no_R2 * w0**2))
print(f"C_filter: {C_filter*1e12:.2f} pF")

DC-Attenuation: 0.9975062344139651
LC-product: 1.1286054066827067e-16
L_filter: 30052.55 nH or 30090.11 nH
C_filter: 3.75 pF


### Thoughts on output filter design
ADC's may have dynamic or static input stages, depending on the way they are designed. Static input stages are easier to design the external front-end for. Dynamic ones are much tougher.

The sample-and-hold structure the pipelined ADC has sees alternating low and high impedance.

When designing an output filter for an ADC, you need to terminate the line at 100 ohms, to make sure the load impedance seems a 100 ohms to the signal.

This needs to be done over the whole frequency band (100 kHz - 10 MHz) in order to minimize reflections.

The issue is apart from that that we have 2 different paths for the ADC signal inputs
- One with magnetics (where the impedance is supposed to be 1:1).
- One without magnetics and with 100 nF blocking capacitors in series.

This 100 nF will again have an effect on how the intermediate BP / LP filter will need to look.

### Chebyshev vs Butterworth
#### Chebyshev
- Main ideal: steep roll-off at the expense of a less-flat passband frequency response.
	- Type 1 Chebyshev: ripples in passband
	- Type 2 Chebyshev: ripples in stop-band

[Link](https://resources.pcb.cadence.com/blog/about-the-chebyshev-filter)
#### Butterworth
- Main idea: Flat frequency response in the passband (maximally flat magnitude filter) at the expense of wider transition bands.
- Quality factor Q = 0.707
- Passive: Cauer topology (LC ladder)
- Active: Sallen-Key topology
- Digital: 
	- matched z-transform 
	- bilinear transform

[Link](https://www.elprocus.com/butterworth-filter-formula-and-calculations/)

### Use one of the Chebyshev / Butterworth filters to simulate a basic input stage filter

### Basic 50-ohm Chebyshev simulation
